In [1]:
import pandas as pd 
import numpy as np
anime=pd.read_csv('anime.csv')
anime.genre.replace(np.nan,'1',inplace=True)
anime.genre.isnull().sum()
anime.head(4)

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572


In [2]:
print(anime.shape)
print(anime.name.nunique())

(12294, 7)
12292


In [3]:
anime.name.value_counts()

Saru Kani Gassen                                     2
Shi Wan Ge Leng Xiaohua                              2
Ingoku Byoutou                                       1
Iinari! Saimin Kanojo                                1
Juushin Liger                                        1
                                                    ..
Magical★Taruruuto-kun: Sukisuki Takoyaki!            1
Crayon Shin-chan Gaiden: Alien vs. Shinnosuke        1
Cynical Hysterie Hour: Trip Coaster                  1
Wagahai wa Inu de Aru: Don Matsugorou no Seikatsu    1
Hakubutsushi                                         1
Name: name, Length: 12292, dtype: int64

In [4]:
anime[(anime['name']=='Shi Wan Ge Leng Xiaohua') | (anime['name']=='Saru Kani Gassen')]

,anime_id,name,genre,type,episodes,rating,members
10140,22399,Saru Kani Gassen,Kids,OVA,1,5.23,62
10141,30059,Saru Kani Gassen,Drama,Movie,1,4.75,76
10193,33193,Shi Wan Ge Leng Xiaohua,"Comedy, Parody",ONA,12,6.67,114
10194,33195,Shi Wan Ge Leng Xiaohua,"Action, Adventure, Comedy, Fantasy, Parody",Movie,1,7.07,110


In [5]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

anime_genre=anime.join(pd.DataFrame(mlb.fit_transform(anime.pop('genre').str.split(',')),columns=mlb.classes_,index=anime.index))
del anime_genre['1']
anime_genre.head(2)

,anime_id,name,type,episodes,rating,members,Adventure,Cars,Comedy,Dementia,...,Shoujo,Shounen,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi
0,32281,Kimi no Na wa.,Movie,1,9.37,200630,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5114,Fullmetal Alchemist: Brotherhood,TV,64,9.26,793665,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
anime_dummy=pd.get_dummies(anime_genre,columns=['type','type']).set_index('anime_id')
anime_dummy.drop(columns=['members','episodes','rating','name'],inplace=True)
anime_dummy.head(3)

,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,...,type_ONA,type_OVA,type_Special,type_TV,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
anime_id,,,,,,,,,,,,,,,,,,,,,
32281,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5114,1,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
28977,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


<h2>TF-IDF</h2>
<blockquote>Giving weight to each attributes by computing tf-idf

In [ ]:
#TF
anime_tf=anime_dummy.applymap(lambda x: x/anime_dummy.sum(axis=1))
anime_tf.head(3)

In [ ]:
#IDF
freq=anime_dummy.sum(axis=0)
idf=(len(anime_dummy)/freq).apply(np.log2)

#weight
weight=anime_tf.mul(idf.values)

<h2>User-item matrix</h2>
<blockquote>By pivoting rating table.

In [ ]:
rating=pd.read_csv('rating.csv')
print('mean ',rating.rating.mean())
print('median ',rating.rating.quantile(0.5))
print('lower boundary ',rating.rating.min())
print('upper boundary',rating.rating.max())
rating.head(5)

In [ ]:
rating_df=pd.pivot_table(rating,values='rating',columns='user_id',index='anime_id')
rating_df

<h3>Computing User preference</h3>

In [ ]:
users=input('userid')
user_metric=pd.DataFrame(columns=anime_tf)

temp=anime_tf.mul(rating.loc[customer],axis=0)
user_metric.loc[customer]=temp.mean(axis=0)
